# Setup

Imports

In [389]:
import pandas as pd
import numpy as np
import sqlite3
import os
import json

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf


Create Connection

In [3]:
conn = sqlite3.connect(os.path.join('C:\\', 'Users', 'Owner', 'Documents', 'Moneyball', 'data', 'data.db'))
c = conn.cursor()

wOBA Weights

In [ ]:
weights = {
    'single':0.884,
    'double':1.261,
    'triple':1.601,
    'home_run':2.072,
    'walk':0.689,
    'hit_by_pitch':0.720,
    'field_error':0,
    'field_out':0,
    'strikeout':0,
    'fielders_choice':0,
    'fielders_choice_out':0,
    'double_play':0,
    'grounded_into_double_play':0,
    'strikeout_double_play':0,
    'triple_play':0,
    
}

Team Colors

In [542]:
team_colors = pd.read_csv('../website/static/team_colors.csv')

,team,c1,c2
0,AZ,#A71930,#E3D4AD
1,ATL,#13274F,#CE1141
2,CHC,#0E3386,#CC3433
3,CIN,#C6011F,#000000
4,COL,#333366,#C4CED4
5,LAD,#005A9C,#FFFFFF
6,MIA,#00A3E0,#EF3340
7,MIL,#12284B,#FFC52F
8,NYM,#002D72,#FF5910
9,PIT,#FDB827,#000000


# Total Shifting Impacts

Shifting Frequency

In [240]:
df_shift_total = pd.read_sql("""
SELECT if_fielding_alignment AS 'Defensive Positioning', (COUNT(if_fielding_alignment)*100.0/(SELECT COUNT(if_fielding_alignment) FROM data22)) AS Frequency
FROM data22
GROUP BY if_fielding_alignment
HAVING Frequency>0
""", conn)

df_shift_total['Frequency'] = df_shift_total['Frequency'].round(decimals=0)

In [385]:
fig_shift_total = px.pie(
    df_shift_total,
    values='Frequency',
    names='Defensive Positioning',
    color_discrete_sequence=['#7BA8E4', '#975a2f', '#2f5597'],
    category_orders={'Defensive Positioning':['Infield shift', 'Strategic', 'Standard']},
    hole=0.4,
    title='% of Pitches Thrown by Defensive Alignment'
)

fig_shift_total.update_layout(
    title_x=0.5,
    title_font_size=18,
    title_font_color='black',
    font_family='Arial Black',
    font_size=14,
    showlegend=False
)

fig_shift_total.update_traces(
    textinfo='percent+label',
    textposition='inside',
    insidetextorientation='horizontal',
    marker_line_color='#ffc000',
    marker_line_width=5,
    textfont_color='white'
)

wOBA by Defensive Alignment

In [146]:
df_shift_woba = pd.read_sql("""
SELECT if_fielding_alignment AS 'Defensive Positioning', AVG(woba_value) AS wOBA
FROM data22
GROUP BY if_fielding_alignment
""", conn)

In [384]:
fig_shift_woba = px.bar(
    df_shift_woba,
    x='Defensive Positioning',
    y='wOBA',
    color='Defensive Positioning',
    color_discrete_sequence=['#2f5597', '#7BA8E4', '#975a2f'],
    category_orders={'Defensive Positioning':['Standard', 'Infield shift', 'Strategic']},
    text='wOBA',
    title='wOBA by Defensive Alignment'
)

fig_shift_woba.update_layout(
    title_x=0.5,
    title_font_size=18,
    title_font_color='black',
    font_family='Arial Black',
    font_size=14,
    plot_bgcolor='white',
    yaxis_tickformat='.3f',
    xaxis_title=None,
    showlegend=False
)

fig_shift_woba.update_traces(
    textposition='inside',
    marker_line_color='#ffc000',
    marker_line_width=5,
    texttemplate='%{text:.3f}',
    insidetextanchor='middle',
    textfont_color='white'
)

fig_shift_woba

wOBA Splits by Batting Hand

In [282]:
df_shift_woba_hand = pd.read_sql("""
SELECT if_fielding_alignment AS 'Defensive Positioning', stand AS Stand, AVG(woba_value) AS wOBA
FROM data22
GROUP BY if_fielding_alignment, stand
""", conn)

df_shift_woba_hand['Stand'] = df_shift_woba_hand['Stand'].map({'L':'Lefty', 'R':'Righty'})

In [383]:
fig_shift_woba_hand = px.bar(
    df_shift_woba_hand,
    x='Defensive Positioning',
    y='wOBA',
    color='Stand',
    color_discrete_sequence=['#2f5597', '#7BA8E4', '#975a2f'],
    category_orders={'Defensive Positioning':['Standard', 'Infield shift', 'Strategic']},
    text='wOBA',
    title='wOBA by Defensive Alignment and Batting Hand',
    barmode='group'
)

fig_shift_woba_hand.update_layout(
    title_x=0.5,
    title_font_size=18,
    title_font_color='black',
    font_family='Arial Black',
    font_size=14,
    plot_bgcolor='white',
    yaxis_tickformat='.3f',
    xaxis_title=None,
    legend_orientation='h',
    legend_y=-.1,
    legend_x=.5,
    legend_xanchor='center'
)

fig_shift_woba_hand.update_traces(
    textposition='inside',
    marker_line_color='#ffc000',
    marker_line_width=5,
    texttemplate='%{text:.3f}',
    insidetextanchor='end',
    textfont_color='white'
)

fig_shift_woba_hand

Shifting Frequency by Batter

In [376]:
df_shift_lefty = pd.read_sql("""
SELECT if_fielding_alignment AS 'Defensive Positioning', (COUNT(if_fielding_alignment)*100.0/(SELECT COUNT(if_fielding_alignment) FROM data22 WHERE stand='L')) AS Frequency
FROM data22
WHERE stand='L'
GROUP BY if_fielding_alignment
HAVING Frequency>0
""", conn)

df_shift_righty = pd.read_sql("""
SELECT if_fielding_alignment AS 'Defensive Positioning', (COUNT(if_fielding_alignment)*100.0/(SELECT COUNT(if_fielding_alignment) FROM data22 WHERE stand='R')) AS Frequency
FROM data22
WHERE stand='R'
GROUP BY if_fielding_alignment
HAVING Frequency>0
""", conn)

df_shift_lefty['Frequency'] = df_shift_lefty['Frequency'].round(decimals=0)
df_shift_righty['Frequency'] = df_shift_righty['Frequency'].round(decimals=0)

In [391]:
# create plots
fig_shift_freq_l_r = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'domain'}, {'type':'domain'}]],
    horizontal_spacing=0.02)

# Lefty
fig_shift_freq_l_r.add_trace(go.Pie(
    labels=df_shift_lefty['Defensive Positioning'],
    values=df_shift_lefty['Frequency'],
    marker_colors=['#7BA8E4', '#2f5597', '#975a2f'],
    marker_line_color='#ffc000',
    marker_line_width=5,
    textinfo='percent+label',
    textposition='inside',
    textfont_color='white',
    textfont_family='Arial Black',
    textfont_size=14,
    hole=0.4,
    title='Lefty<br>Batters',
    title_font_family='Arial Black',
    title_font_color='black',
    title_font_size=18,
), 1, 1)

# Righty
fig_shift_freq_l_r.add_trace(go.Pie(
    labels=df_shift_righty['Defensive Positioning'],
    values=df_shift_righty['Frequency'],
    marker_colors=['#7BA8E4', '#2f5597', '#975a2f'],
    marker_line_color='#ffc000',
    marker_line_width=5,
    textinfo='percent+label',
    textposition='inside',
    textfont_color='white',
    textfont_family='Arial Black',
    textfont_size=14,
    hole=0.4,
    title='Righty<br>Batters',
    title_font_family='Arial Black',
    title_font_color='black',
    title_font_size=18,
    sort=False
), 1, 2)

fig_shift_freq_l_r.update_layout(
    title_text='% of Pitches Thrown by Defensive Alignment',
    title_font_family='Arial Black',
    title_font_color='Black',
    title_x=0.5,
    showlegend=False)

Shifting Frequency by Pitching Team

In [544]:
df_shift_freq_pitch_team = pd.read_sql("""

SELECT
    og.pitcher_team AS 'Pitching Team',
    og.if_fielding_alignment AS 'Defensive Positioning',
    ROUND(COUNT(og.if_fielding_alignment)*100.0 / help.Count, 1) AS Percent
    
FROM data22 og LEFT JOIN
    (
        SELECT
            pitcher_team,
            COUNT(if_fielding_alignment) AS Count
        FROM data22
        GROUP BY pitcher_team
    ) help ON og.pitcher_team=help.pitcher_team
    
GROUP BY og.pitcher_team, og.if_fielding_alignment

HAVING og.if_fielding_alignment NOT NULL

""", conn)

df_shift_freq_pitch_team = df_shift_freq_pitch_team.pivot(index='Pitching Team', columns='Defensive Positioning', values='Percent').reset_index()
df_shift_freq_pitch_team = df_shift_freq_pitch_team.rename_axis(None, axis=1)
df_shift_freq_pitch_team['Any Shift'] = 100-df_shift_freq_pitch_team['Standard']
df_shift_freq_pitch_team = df_shift_freq_pitch_team.sort_values(by='Any Shift', ascending=True)
df_shift_freq_pitch_team = df_shift_freq_pitch_team.merge(team_colors, left_on='Pitching Team', right_on='team')

In [553]:
fig_shift_freq_pitch_team = go.Figure()

fig_shift_freq_pitch_team.add_trace(go.Bar(
    y=df_shift_freq_pitch_team['Pitching Team'],
    x=df_shift_freq_pitch_team['Infield shift'],
    name='Infield Shift',
    orientation='h',
    marker_color=df_shift_freq_pitch_team['c1']
))

fig_shift_freq_pitch_team.add_trace(go.Bar(
    y=df_shift_freq_pitch_team['Pitching Team'],
    x=df_shift_freq_pitch_team['Strategic'],
    name='Strategic',
    orientation='h',
    marker_color=df_shift_freq_pitch_team['c2']
))

fig_shift_freq_pitch_team.update_layout(
    height=800,
    title='Shifting Frequency by Pitching Team',
    title_font_family='Arial Black',
    title_font_color='black',
    plot_bgcolor='white',
    barmode='stack',
    bargap=0.1,
    title_x=0.5
)

fig_shift_freq_pitch_team.update_yaxes(
    tickfont_family='Arial Black',
    tickfont_color='black'
)

Shifting Frequency by Batting Team

In [532]:
df_shift_freq_bat_team = pd.read_sql("""

SELECT
    og.batter_team AS 'Batting Team',
    og.if_fielding_alignment AS 'Defensive Positioning',
    ROUND(COUNT(og.if_fielding_alignment)*100.0 / help.Count, 1) AS Percent
    
FROM data22 og LEFT JOIN
    (
        SELECT
            batter_team,
            COUNT(if_fielding_alignment) AS Count
        FROM data22
        GROUP BY batter_team
    ) help ON og.batter_team=help.batter_team
    
GROUP BY og.batter_team, og.if_fielding_alignment

HAVING og.if_fielding_alignment NOT NULL

""", conn)

df_shift_freq_bat_team = df_shift_freq_bat_team.pivot(index='Batting Team', columns='Defensive Positioning', values='Percent').reset_index()
df_shift_freq_bat_team = df_shift_freq_bat_team.rename_axis(None, axis=1)
df_shift_freq_bat_team['Any Shift'] = 100-df_shift_freq_bat_team['Standard']
df_shift_freq_bat_team = df_shift_freq_bat_team.sort_values(by='Any Shift', ascending=True)

In [554]:
fig_shift_freq_bat_team = go.Figure()

fig_shift_freq_bat_team.add_trace(go.Bar(
    y=df_shift_freq_bat_team['Batting Team'],
    x=df_shift_freq_bat_team['Infield shift'],
    name='Infield Shift',
    orientation='h',
    marker_color='#2f5597'
))

fig_shift_freq_bat_team.add_trace(go.Bar(
    y=df_shift_freq_bat_team['Batting Team'],
    x=df_shift_freq_bat_team['Strategic'],
    name='Strategic',
    orientation='h',
    marker_color='#7BA8E4'
))

fig_shift_freq_bat_team.update_layout(
    height=800,
    title='Shifting Frequency by Batting Team',
    title_font_family='Arial Black',
    title_font_color='black',
    plot_bgcolor='white',
    barmode='stack',
    bargap=0.1,
    title_x=0.5
)

fig_shift_freq_bat_team.update_yaxes(
    tickfont_family='Arial Black',
    tickfont_color='black'
)

Shifting wOBA by Pitching Team

Shifting wOBA by Batting Team

Shifting Frequency by Pitcher

Shifting Frequency by Batter

Shifting wOBA by Pitcher

Shifting wOBA by Batter

# Save Figures to JSON

In [558]:
fig_dict = {}
fig_dict['fig_shift_total'] = fig_shift_total.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})
fig_dict['fig_shift_woba'] = fig_shift_woba.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})
fig_dict['fig_shift_woba_hand'] = fig_shift_woba_hand.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})
fig_dict['fig_shift_freq_l_r'] = fig_shift_freq_l_r.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})
fig_dict['fig_shift_freq_pitch_team'] = fig_shift_freq_pitch_team.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})
fig_dict['fig_shift_freq_bat_team'] = fig_shift_freq_bat_team.to_html(full_html=False, config={'displayModeBar':False, 'responsive':True})

with open(os.path.join('C:\\', 'Users', 'Owner', 'Documents', 'Moneyball', 'website', 'static', 'saved_figs.json'), "w") as f:
    json.dump(fig_dict, f)

In [6]:
df = pd.read_sql("SELECT * FROM data22 WHERE batter_name='Kris Bryant'", conn)
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'batter', 'pitcher', 'events', 'description',
       'spin_dir', 'zone', 'game_type', 'stand', 'p_throws', 'home_team',
       'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes',
       'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b',
       'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
       'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
       'post_home_score', 'post_bat_s

In [ ]:
conn.close()